<a href="https://colab.research.google.com/github/Dimas0824/Machine_Learning/blob/main/Jobsheet_6/Week7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Praktikum 3

In [11]:
# Install hnswlib terlebih dahulu.
!pip install hnswlib


  Using cached hnswlib-0.8.0.tar.gz (36 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp312-cp312-linux_x86_64.whl size=2528145 sha256=240b68013e9d5ba0520ea7e550acf605c3c56370e9b88415ab1a0c7e1b8f90fe
  Stored in directory: /root/.cache/pip/wheels/ac/39/b3/cbd7f9cbb76501d2d5fbc84956e70d0b94e788aac87bda465e
Successfully built hnswlib


Percobaan berikut akan membandingkan exact NN dengan HNSW pada 1000 data 2D.

In [14]:
import hnswlib
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors

# ===========================
# 1. Buat data 2D acak
# ===========================
num_elements = 1000
dim = 2
data = np.random.random((num_elements, dim)).astype(np.float32)

# Query point
query = np.array([[0.5, 0.5]], dtype=np.float32)
k = 5  # cari 5 tetangga terdekat

# ===========================
# 2. Exact NN (Brute Force)
# ===========================
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(data)

start = time.time()
distances, indices = nn.kneighbors(query)
end = time.time()

print("=== Exact NN ===")
print("Indices:", indices)
print("Distances:", distances)
print("Waktu:", end - start, "detik")

# ===========================
# 3. HNSW
# ===========================
# Inisialisasi index HNSW
p = hnswlib.Index(space='l2', dim=dim)

# Ukuran maksimum elemen yang bisa ditampung
p.init_index(max_elements=num_elements, ef_construction=100, M=16)

# Tambahkan data
p.add_items(data)

# Set parameter pencarian
p.set_ef(50)   # tradeoff speed vs accuracy

start = time.time()
labels, distances = p.knn_query(query, k=k)
end = time.time()

print("\n=== HNSW ===")
print("Indices:", labels)
print("Distances:", distances)
print("Waktu:", end - start, "detik")


=== Exact NN ===
Indices: [[636 833 104 745 565]]
Distances: [[0.00378809 0.03173314 0.03653261 0.04707944 0.05141171]]
Waktu: 0.0016665458679199219 detik

=== HNSW ===
Indices: [[636 833 104 745 565]]
Distances: [[1.4349619e-05 1.0069924e-03 1.3346312e-03 2.2164732e-03 2.6431645e-03]]
Waktu: 0.00020051002502441406 detik


Lakukan percobaan pada metric distance yang berbeda, 1000 vs 1jt data, 2D vs 5D data. catat hasilnya pada tabel yang anda buat sendiri seperti pada praktikum 1.

| Jumlah Data | Dimensi | Metode | Indices | Distances | Waktu (detik) |
|--------------|----------|---------|----------|-------------|----------------|
| 1000 | 2D | Exact NN (Flat) | [[636 833 104 745 565]] | [[0.00378809 0.03173314 0.03653261 0.04707944 0.05141171]] | 0.0017 |
| 1000 | 2D | HNSW (Approx) | [[636 833 104 745 565]] | [[1.4349619e-05 1.0069924e-03 1.3346312e-03 2.2164732e-03 2.6431645e-03]] | 0.0002 |
| 1.000.000 | 5D | Exact NN (Brute Force) | [[598419 133481 936011 383085 427107]] | [[0.04274838 0.04917056 0.05724072 0.05813525 0.06194251]] | 0.1374 |
| 1.000.000 | 5D | HNSW (Approx) | [[598419 133481 936011 383085 427107]] | [[0.00182742 0.00241774 0.0032765  0.00337971 0.00383687]] | 0.0004 |



In [15]:
import hnswlib
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors

# ==========================================================
# 1. Generate Data 1 Juta Sampel Berdimensi 5 (5D)
# ==========================================================
num_elements = 1_000_000   # jumlah data
dim = 5                    # jumlah dimensi
data = np.random.random((num_elements, dim)).astype(np.float32)

# Titik query tunggal (bisa dari data atau titik acak)
query = np.random.random((1, dim)).astype(np.float32)
k = 5  # cari 5 tetangga terdekat

# ==========================================================
# 2. Exact Nearest Neighbor (Brute Force)
# ==========================================================
# Gunakan sklearn NearestNeighbors dengan algoritma brute-force (akurasi 100%)
print("=== Exact NN (Brute Force) ===")
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(data)

start = time.time()
distances_exact, indices_exact = nn.kneighbors(query)
end = time.time()
exact_time = end - start

print("Indices:", indices_exact)
print("Distances:", distances_exact)
print("Waktu Exact NN :", round(exact_time, 4), "detik")

# ==========================================================
# 3. Approximate NN dengan HNSW
# ==========================================================
# Inisialisasi index HNSW (Hierarchical Navigable Small World Graph)
print("\n=== Approx NN (HNSW) ===")
p = hnswlib.Index(space='l2', dim=dim)  # space 'l2' = Euclidean distance

# Inisialisasi struktur index
p.init_index(max_elements=num_elements, ef_construction=100, M=16)

# Tambahkan data ke index (proses build index)
print("Membangun index HNSW...")
p.add_items(data)

# Set ef parameter untuk pencarian (tradeoff antara akurasi & kecepatan)
p.set_ef(100)

# Lakukan query ANN
start = time.time()
labels_ann, distances_ann = p.knn_query(query, k=k)
end = time.time()
ann_time = end - start

print("Indices:", labels_ann)
print("Distances:", distances_ann)
print("Waktu Approx NN (HNSW):", round(ann_time, 4), "detik")

# ==========================================================
# 4. Ringkasan Hasil Perbandingan
# ==========================================================
print("\n=== RINGKASAN PERBANDINGAN ===")
print(f"Jumlah data\t\t: {num_elements}")
print(f"Dimensi data\t\t: {dim}D")
print(f"Waktu Exact NN\t\t: {exact_time:.4f} detik")
print(f"Waktu HNSW ANN\t\t: {ann_time:.4f} detik")


=== Exact NN (Brute Force) ===
Indices: [[598419 133481 936011 383085 427107]]
Distances: [[0.04274838 0.04917056 0.05724072 0.05813525 0.06194251]]
Waktu Exact NN : 0.1374 detik

=== Approx NN (HNSW) ===
Membangun index HNSW...
Indices: [[598419 133481 936011 383085 427107]]
Distances: [[0.00182742 0.00241774 0.0032765  0.00337971 0.00383687]]
Waktu Approx NN (HNSW): 0.0004 detik

=== RINGKASAN PERBANDINGAN ===
Jumlah data		: 1000000
Dimensi data		: 5D
Waktu Exact NN		: 0.1374 detik
Waktu HNSW ANN		: 0.0004 detik
